In [1]:
###To see the dashboard, run the code, then copy/paste "http://127.0.0.1:8056/" into your browser

######### Importing Libraries ###########################################

import dash
from dash import dcc
from dash import html
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import date
import plotly.graph_objects as go
from datetime import datetime
from dash import dash_table

######### Importing Data #############################################
xls = pd.ExcelFile('Dash.xlsx',engine='openpyxl')    ## Make sure to rename your file name or change "ML2" to your excel file 
generator = []   ## Loads generators names (excel sheets names)
data = {}  #Blank dictionary to store all data included in the excel file

# Preload all the necessary sheets into a cache (dictionary)
for sheet in xls.sheet_names:
    generator.append(sheet)
    data[sheet] = pd.read_excel(xls, sheet_name=sheet)
    data[sheet].set_index('Date',inplace=True)
dropdown_options = [{'label': sheet, 'value': sheet} for sheet in generator]
results = {}

############### APP LAYOUT #############################################
app = dash.Dash()
app.layout = html.Div([
    # Header that spans the entire width
    html.H3('Generator Temperature Prediction Dashboard', style={
        'width': '98%',
        'padding': '1rem',
        'margin': '0',
        'border-radius': '15px',
        #'backgroundColor': '#2E3B55',  # Dark navy header
        'backgroundColor': '#36454F',
        'color': 'white',  # White text for contrast
        'textAlign': 'center',
        'fontSize': '2.5rem',
        'fontFamily': 'Arial, Helvetica, sans-serif'
    }),

    # Flex container for the entire layout
    html.Div([

        # Flex container for the left side (first generator + current controls)
        html.Div([
            # First generator box (placed above the current boxes)
            html.Div([
                html.Div('Select Generator:', style={
                    'fontSize': 24, 
                    'fontFamily': 'Arial, Helvetica, sans-serif', 
                    'color': '#2E3B55'  # Dark navy text
                }),
                dcc.Dropdown(id='dropdown',
                             options=dropdown_options,
                             value=None,
                             style={
                                 'verticalAlign': 'top',
                                 'width': '90%',
                                 'padding': '10px',
                                 'fontSize': 26,
                                 'color': '#2E3B55'  # Text color for dropdown
                             })
            ], style={
                'width': '100%',
                'marginBottom': '1rem',
                'border': '2px solid #B0BEC5',  # Light gray border
                'border-radius': '10px',
                'padding': '1rem',
                'box-sizing': 'border-box',
                'backgroundColor': '#F0F0F0'  # Light gray background
            }),

            # current controls (placed below the first generator box)
            html.Div([
                html.H2('Model Parameters', style={
                    'fontFamily': 'Arial, Helvetica, sans-serif',
                    'textAlign': 'left',
                    'color': '#2E3B55',
                    'marginTop': '0px',
                    'paddingTop': '0px'
                }),
                # Select current section 1
                html.Div([
                    html.Div('Select electrical current:', style={
                        'fontFamily': 'Arial, Helvetica, sans-serif',
                        'fontSize': '1.5rem',
                        'color': '#2E3B55',
                        'marginBottom': '10px'
                    }),
                    dcc.Dropdown(
                        id="current1",
                        options=[{'label': 'Option 1', 'value': '1'},
                                 {'label': 'Option 2', 'value': '2'},
                                 {'label': 'Option 3', 'value': '3'}],
                        style={'width': '100%', 'marginBottom': '1rem', 'fontSize': '22px'}
                    ),
                    dcc.RangeSlider(
                        id="current",
                        step=0.1,
                        marks={0: {'label': '0', 'style': {'fontSize': '20px', 'color': '#2E3B55'}},
                               100: {'label': '100', 'style': {'fontSize': '20px', 'color': '#2E3B55'}}},
                        value=[0.5, 50],
                        tooltip={"placement": "bottom", "always_visible": False}
                    )
                ], style={
                    'width': '100%',
                    'box-sizing': 'border-box',
                    'border-radius': '10px',
                    'paddingbottom': '5px',
                    'marginBottom': '2rem'
                }),

                # Select current section 2
                html.Div([
                    html.Div('Select ambient temperature:', style={
                        'fontFamily': 'Arial, Helvetica, sans-serif',
                        'fontSize': '1.5rem',
                        'color': '#2E3B55',
                        'marginBottom': '10px',
                        'margintop': '10px'
                    }),
                    dcc.Dropdown(
                        id="Temp1",
                        options=[{'label': 'Option 1', 'value': '1'},
                                 {'label': 'Option 2', 'value': '2'},
                                 {'label': 'Option 3', 'value': '3'}],
                        style={'width': '100%', 'marginBottom': '1rem', 'fontSize': '22px'}
                    ),
                    dcc.RangeSlider(
                        id="Temp",
                        step=0.1,
                        marks={0: {'label': '0', 'style': {'fontSize': '20px', 'color': 'red'}},
                               100: {'label': '100', 'style': {'fontSize': '20px', 'color': 'red'}}},
                        value=[0.5, 50],
                        tooltip={"placement": "bottom", "always_visible": False}
                    )
                ], style={
                    'width': '100%',
                    'box-sizing': 'border-box',
                    'border-radius': '10px',
                    'marginBottom': '2rem'
                }),

                # Select current section 3
                html.Div([
                    html.Div('Select Temperature sensor:', style={
                        'fontFamily': 'Arial, Helvetica, sans-serif',
                        'fontSize': '1.5rem',
                        'color': '#2E3B55',
                        'marginBottom': '10px'
                    }),
                    dcc.Dropdown(
                        id="Tag1",
                        options=[{'label': 'Option 1', 'value': '1'},
                                 {'label': 'Option 2', 'value': '2'},
                                 {'label': 'Option 3', 'value': '3'}],
                        style={'width': '100%', 'marginBottom': '1rem', 'fontSize': '22px'}
                    ),
                    dcc.RangeSlider(
                        id="Tag",
                        step=0.1,
                        marks={0: {'label': '0', 'style': {'fontSize': '20px', 'color': '#2E3B55'}},
                               100: {'label': '100', 'style': {'fontSize': '20px', 'color': '#2E3B55'}}},
                        value=[0.5, 50],
                        tooltip={"placement": "bottom", "always_visible": False}
                    )
                ], style={
                    'width': '100%',
                    'box-sizing': 'border-box',
                    'border-radius': '10px',
                })
            ], style={
                'width': '100%',
                'height': '100%',
                'border': '2px solid #B0BEC5',  # Light gray border
                'border-radius': '10px',
                'padding': '1rem',
                'box-sizing': 'border-box',
                'backgroundColor': '#F0F0F0',  # Light gray background
                'flex-grow': '1'
            })
        ], style={
            'width': '20%',
            'box-sizing': 'border-box',
            'flex-grow': '1',
            'height': '100%'
        }),

        # Right box for Machine Learning Model Plot and Date Selection
        html.Div([

            html.Div([
                html.H2('Prediction vs Actual Plot', style={
                    'fontFamily': 'Arial, Helvetica, sans-serif',
                    'textAlign': 'left',
                    'color': '#2E3B55',
                    'marginTop': '-0.2rem'
                }),
                html.Div(dcc.Graph(id='plot'), style={
                    'marginTop': '0rem',
                    'width': '100%',
                    'border-radius': '10px',
                    'marginBottom': '2.5rem'
                })
            ]),

            # Flexbox for the three date picker boxes and buttons
            html.Div([

                # Start Date / End Date Box
                html.Div([
                    html.Div('Start date/End date', style={
                        'fontFamily': 'Arial, Helvetica, sans-serif',
                        'fontSize': '1.5rem',
                        'color': '#2E3B55'
                    }),
                    dcc.DatePickerRange(
                        id='calendar',
                        min_date_allowed='2019-01-01',
                        max_date_allowed=datetime.now().strftime('%Y-%m-%d'),
                        start_date=datetime(2020, 1, 1),
                        end_date=datetime.now().strftime('%Y-%m-%d'),
                        style={
                            'padding': '0.5rem',
                            'width': '100%',
                            'border-radius': '5px',
                            'fontSize': '1.6rem'
                        }
                    )
                ], style={
                    'marginBottom': '0.1rem',
                    'width': '40%',
                    'box-sizing': 'border-box',
                    'border': '2px solid #B0BEC5',  # Light gray border
                    'padding': '1rem',
                    'border-radius': '10px',
                    'backgroundColor': '#F0F0F0'  # Light gray background
                }),

                # Training Data Box
                html.Div([
                    html.Div('Training data', style={
                        'fontFamily': 'Arial, Helvetica, sans-serif',
                        'fontSize': '1.5rem',
                        'color': '#2E3B55'
                    }),
                    dcc.DatePickerRange(
                        id='Training',
                        min_date_allowed='2020-01-01',
                        max_date_allowed=datetime.now().strftime('%Y-%m-%d'),
                        start_date=datetime(2020, 1, 1),
                        end_date=datetime(2023, 1, 1),
                        style={
                            'padding': '0.5rem',
                            'width': '100%',
                            'border-radius': '5px',
                            'fontSize': '1.6rem'
                        }
                    )
                ], style={
                    'marginBottom': '0.1rem',
                    'width': '40%',
                    'box-sizing': 'border-box',
                    'border': '2px solid #B0BEC5',
                    'padding': '1rem',
                    'border-radius': '10px',
                    'backgroundColor': '#F0F0F0'
                }),

                # Testing Data Box
                html.Div([
                    html.Div('Testing data', style={
                        'fontFamily': 'Arial, Helvetica, sans-serif',
                        'fontSize': '1.5rem',
                        'color': '#2E3B55'
                    }),
                    dcc.DatePickerRange(
                        id='Testing',
                        min_date_allowed='2019-01-01',
                        max_date_allowed=datetime.now().strftime('%Y-%m-%d'),
                        start_date=datetime(2023, 1, 1),
                        end_date=datetime.now().strftime('%Y-%m-%d'),
                        style={
                            'padding': '0.5rem',
                            'width': '100%',
                            'border-radius': '5px',
                            'fontSize': '1.6rem'
                        }
                    )
                ], style={
                    'marginBottom': '0.1rem',
                    'width': '40%',
                    'box-sizing': 'border-box',
                    'border': '2px solid #B0BEC5',
                    'padding': '1rem',
                    'border-radius': '10px',
                    'backgroundColor': '#F0F0F0'
                }),

                # Stacked Update/Reset buttons next to the Testing Data Box
                html.Div([
                    html.Button('Results Graph', id='update-button', style={
                        'font-size': '24px',
                        'padding': '10px 20px',
                        'width': '200px',
                        'height': '60px',
                        'border-radius': '10px',
                        'background-color': '#2E8B57',  # Forest green for action
                        'color': 'white',
                        'margin-top': '-10px'
                    }, n_clicks=0),

                    html.Button('Reset Graph', id='reset-button', style={
                        'font-size': '24px',
                        'padding': '10px 20px',
                        'width': '200px',
                        'height': '60px',
                        'border-radius': '10px',
                        'background-color': '#8B0000',  # Dark red for reset
                        'color': 'white',
                        'margin-top': '5px'
                    }, n_clicks=0)
                ], style={
                    'display': 'flex',
                    'flex-direction': 'column',  # Stack buttons vertically
                    'align-items': 'center',
                    'margin-top': '10px'
                })
            ], style={
                'display': 'flex',
                'width': '100%',
                'box-sizing': 'border-box',
                'gap': '10px',
                'margintop': '5rem'
            })
        ], style={
            'width': '80%',
            'border': '2px solid #B0BEC5',  # Light gray border
            'border-radius': '10px',
            'padding': '1rem',
            'box-sizing': 'border-box',
            'flex-grow': '1'
        })
    ], style={
        'display': 'flex',
        'width': '100%',
        'padding': '1rem',
        'box-sizing': 'border-box',
        'gap': '10px',
        'backgroundColor': '#F5F5F5'  # Light gray for overall background
    })
])




########### CALLBACK ##########################################

@app.callback(
    Output('current1', 'options'),
    Input('dropdown', 'value'))

def update_current(value):
    if value in data:
        df = data[value]
        #values = df['Amp'].tolist()
        # Create options for the current dropdown based on the 'Amp' column values
        options = [{'label': str(val), 'value': val} for val in df.columns]
    else:
        # Default values for options, slider value, and marks
        options = [{'label': 'No data available', 'value': ''}]
    return options   #, slider_value, marks


@app.callback(
    [
    Output('current', 'value'),
    Output('current', 'marks')],
    [Input('dropdown', 'value'),
        Input('current1', 'value')]
)

def update_amp_slider(value,value1):
    if value in data and value1 is not None:
        df = data[value]
        values = df[value1].tolist()
        # Create options for the current dropdown based on the 'Amp' column values
    
        # Create marks with min and max values
        slider_value = [int(min(values)), int(max(values))]
        marks = {
            int(min(values)): {'label': str(int(min(values))), 'style': {'font-size': '24px'}},
            int(max(values)): {'label': str(int(max(values))), 'style': {'font-size': '24px'}}
        }
    else:
        # Default values for options, slider value, and marks
        slider_value = [0, 100]
        marks = {0: {'label': '0', 'style': {'font-size': '24px'}},
                 100: {'label': '100', 'style': {'font-size': '24px'}}}
    return slider_value, marks



@app.callback(
    Output('Temp1', 'options'),
    Input('dropdown', 'value')
)

def update_ambient_temp(value):
    if value in data:
        df = data[value]
      
        # Create options for the current dropdown based on the 'Amp' column values
        options = [{'label': str(val), 'value': val} for val in df.columns]
        # Create marks with min and max values
        
    else:
        # Default values for options, slider value, and marks
        options = [{'label': 'No data available', 'value': ''}]
    return options




@app.callback(
    [
    Output('Temp', 'value'),
    Output('Temp', 'marks')],
    [Input('dropdown', 'value'),
        Input('Temp1', 'value')]
)

def update_amp_slider(value,value1):
    if value in data and value1 is not None:
        df = data[value]
        values = df[value1].tolist()
        # Create options for the current dropdown based on the 'Amp' column values
    
        # Create marks with min and max values
        slider_value = [int(min(values)), int(max(values))]
        marks = {
            int(min(values)): {'label': str(int(min(values))), 'style': {'font-size': '24px'}},
            int(max(values)): {'label': str(int(max(values))), 'style': {'font-size': '24px'}}
        }
    else:
        # Default values for options, slider value, and marks
        slider_value = [0, 100]
        marks = {0: {'label': '0', 'style': {'font-size': '24px'}},
                 100: {'label': '100', 'style': {'font-size': '24px'}}}
    return slider_value, marks


@app.callback(
    Output('Tag1', 'options'),
    Input('dropdown', 'value')
)

def update_tags(value):
    if value in data:
        df = data[value]
        #values = df.columns[3:]
        # Create options for the current dropdown based on the 'Amp' column values
        options = [{'label': str(val), 'value': val} for val in df.columns[2:]]
        # Create marks with min and max values
    else:
        # Default values for options, slider value, and marks
        options = [{'label': 'No data available', 'value': ''}]
    return options


@app.callback(
    [
    Output('Tag', 'value'),
    Output('Tag', 'marks')],
    [Input('dropdown', 'value'),
        Input('Tag1', 'value')]
)

def update_tags_slider(value,value1):
    if value in data and value1 is not None:
        df = data[value]
        values = df[value1].tolist()
        # Create options for the current dropdown based on the 'Amp' column values
    
        # Create marks with min and max values
        slider_value = [int(min(values)), int(max(values))]
        marks = {
            int(min(values)): {'label': str(int(min(values))), 'style': {'font-size': '24px'}},
            int(max(values)): {'label': str(int(max(values))), 'style': {'font-size': '24px'}}
        }
    else:
        # Default values for options, slider value, and marks
        slider_value = [0, 100]
        marks = {0: {'label': '0', 'style': {'font-size': '24px'}},
                 100: {'label': '100', 'style': {'font-size': '24px'}}}
    return slider_value, marks



@app.callback(
    Output('plot', 'figure'),
    [
        Input('dropdown', 'value'),  
        Input('Tag1', 'value'),  
        Input('current', 'value'),  
        Input('Temp', 'value'),  
        Input('Tag', 'value'),  
        Input('calendar','start_date'),
        Input('calendar','end_date'),
        Input('Training', 'start_date'),  
        Input('Training', 'end_date'),  
        Input('Testing', 'start_date'),  
        Input('Testing', 'end_date'),  
        Input('update-button', 'n_clicks'),
        Input('reset-button', 'n_clicks')
    ])

def update_graph(generator_value, tag_value, current_range, temp_range, tag_range, calendar_start,calendar_end, train_start, train_end, test_start, test_end, update_clicks, reset_clicks):
    # Get the cached data
    df = data.get(generator_value)
    if df is None or not generator_value or not tag_value:
        return go.Figure()

    df.index = pd.to_datetime(df.index)

    # Check if either the reset button or update button was clicked
    if reset_clicks > 0 or update_clicks > 0:
        if reset_clicks >= update_clicks:
            # Reset to the original graph when the reset button is clicked
            train_data=df[(df.index >= calendar_start) & (df.index < calendar_end)]
            #train_data = df[(df.index >= '2020-01-01') & (df.index < '2024-08-01')]
            train_data = train_data[(train_data['Electrical_current'] >= current_range[0]) & (train_data['Electrical_current'] <= current_range[1])]
            train_data = train_data[(train_data['Ambient_Temperature'] >= temp_range[0]) & (train_data['Ambient_Temperature'] <= temp_range[1])]
            train_data = train_data[(train_data[tag_value] >= tag_range[0]) & (train_data[tag_value] <= tag_range[1])]

            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=train_data.index,
                y=train_data['Electrical_current'],
                name='Electrical_current',
                marker=dict(color='rgb(34,163,192)'),
                line=dict(width=3)
            ))
            fig.add_trace(go.Scatter(
                x=train_data.index,
                y=train_data['Ambient_Temperature'],
                name='Ambient Temperature',
                line=dict(width=3)
            ))
            fig.add_trace(go.Scatter(
                x=train_data.index,
                y=train_data[tag_value],
                name=tag_value,
                marker=dict(color='#32CD32'),
                line=dict(width=3)
            ))

            fig.update_layout(
                plot_bgcolor='white',
                paper_bgcolor='white',
                xaxis=dict(
                    showline=True,
                    linecolor='black',
                    linewidth=1,
                    mirror=True,
                    ticks='outside',
                    tickfont=dict(
                        family='Arial',
                        size=20,
                        color='black'
                    ),
                    showgrid=True,
                    gridcolor='lightgray',
                    gridwidth=1
                ),
                yaxis=dict(
                    showline=True,
                    linecolor='black',
                    linewidth=1,
                    mirror=True,
                    ticks='outside',
                    tickfont=dict(
                        family='Arial',
                        size=20,
                        color='black'
                    ),
                    showgrid=True,
                    gridcolor='lightgray',
                    gridwidth=1
                ),
                font=dict(size=22)
               # height=620
            )
            return fig

        elif update_clicks > reset_clicks:
            # Update the graph based on user input when the update button is clicked
            train_data = df[(df.index >= train_start) & (df.index < train_end)]
            test_data = df[(df.index >= test_start) & (df.index < test_end)]

            train_data = train_data[(train_data['Electrical_current'] >= current_range[0]) & (train_data['Electrical_current'] <= current_range[1])]
            train_data = train_data[(train_data['Ambient_Temperature'] >= temp_range[0]) & (train_data['Ambient_Temperature'] <= temp_range[1])]
            train_data = train_data[(train_data[tag_value] >= tag_range[0]) & (train_data[tag_value] <= tag_range[1])]

            features = df.columns[:2]
            target = tag_value

            model = xgb.XGBRegressor(
                                    learning_rate=0.3, 
                                    max_depth=6, 
                                    n_estimators=100, 
                                    objective='reg:squarederror', 
                                    random_state=0
                                )
            model.fit(X=train_data[features], y=train_data[target])

            pred_test = model.predict(test_data[features])
            pred_test_df = pd.DataFrame(pred_test, index=test_data.index, columns=['prediction'])

            
            diff = test_data[target] - pred_test_df['prediction']
            diff1 = pd.DataFrame(diff, columns=['diff'])
            diff1 = diff1.reset_index()
            negative = diff1[diff1['diff'] < -10]
            negative.set_index('Date', inplace=True)
            points = []
            i = 0
            while i <= len(diff1) - 24:
                window = diff1.iloc[i:i + 24]
                if (window['diff'] > 10).all():
                    points.append(diff1.index[i])
                    i += 24
                else:
                    i += 1

            result_df = diff1.loc[points]
            if not result_df.empty:
                x = []
                for i in range(len(result_df) - 1):
                    if result_df.iloc[i]['Date'].hour != result_df.iloc[i + 1]['Date'].hour:
                        x.append(i + 1)
                x.append(0)
                anomaly = result_df.iloc[x]
                anomaly.set_index('Date', inplace=True)

                if anomaly.empty:
                    anomaly=[]
                result_df.set_index('Date', inplace=True)
    
                
            else:
                result_df=[]
            diff1.set_index('Date', inplace=True)     

            fig = go.Figure()
           # diff1.set_index('Date', inplace=True)
            fig.add_trace(go.Scatter(
                x=pred_test_df.index,
                y=pred_test_df['prediction'],
                name='Prediction',
                marker=dict(color='rgb(34,163,192)'),
                line=dict(width=3)
            ))

            fig.add_trace(go.Scatter(
                x=test_data.index,
                y=test_data[target],
                name='Actual',
                line=dict(width=3)
            ))
            
            fig.add_trace(go.Scatter(
                x=diff1.index,
                y=diff1['diff'],
                name='Residual',
                marker=dict(color='#32CD32'),
                line=dict(width=3)
            ))
            if len(result_df)==0:
                result_df=[]
            else:    
                fig.add_trace(go.Scatter(
                    x=result_df.index,
                    y=result_df['diff'],
                    name='24-hr Anomaly points',
                    mode='markers',
                    marker=dict(size=8, color='black')
            ))

            if 'anomaly' in globals() or 'anomaly' in locals():
                if len(anomaly)==0:
                    anomaly=[]
                else:
                    fig.add_trace(go.Scatter(
                        x=anomaly.index,
                        y=anomaly['diff'],
                        name='Anomaly Onset',
                        mode='markers',
                        marker=dict(color='#D626FF', size=17, line=dict(width=2, color='DarkSlateGrey'))
                    ))

#             fig.add_trace(go.Scatter(
#                 x=negative.index,
#                 y=negative['diff'],
#                 name='Retrain',
#                 mode='markers',
#                 marker=dict(size=5, color='blue')
#             ))

            fig.update_layout(
                plot_bgcolor='white',
                paper_bgcolor='white',
                xaxis=dict(
                    showline=True,
                    linecolor='black',
                    linewidth=1,
                    mirror=True,
                    ticks='outside',
                    tickfont=dict(
                        family='Arial',
                        size=20,
                        color='black'
                    ),
                    showgrid=True,
                    gridcolor='lightgray',
                    gridwidth=1
                ),
                yaxis=dict(
                    showline=True,
                    linecolor='black',
                    linewidth=1,
                    mirror=True,
                    ticks='outside',
                    tickfont=dict(
                        family='Arial',
                        size=20,
                        color='black'
                    ),
                    showgrid=True,
                    gridcolor='lightgray',
                    gridwidth=1
                ),
                font=dict(size=22),
            #   height=620
            )

            return fig
    else:
        # Initial graph load after the tag is selected
        
        train_data=df[(df.index >= calendar_start) & (df.index < calendar_end)]
        #train_data = df[(df.index >= '2020-01-01') & (df.index < '2024-08-01')]
        train_data = train_data[(train_data['Electrical_current'] >= current_range[0]) & (train_data['Electrical_current'] <= current_range[1])]
        train_data = train_data[(train_data['Ambient_Temperature'] >= temp_range[0]) & (train_data['Ambient_Temperature'] <= temp_range[1])]
        train_data = train_data[(train_data[tag_value] >= tag_range[0]) & (train_data[tag_value] <= tag_range[1])]
        target = tag_value

        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=train_data.index,
            y=train_data['Electrical_current'],
            name='Electrical_current',
            marker=dict(color='rgb(34,163,192)'),
            line=dict(width=3)
        ))
        fig.add_trace(go.Scatter(
            x=train_data.index,
            y=train_data['Ambient_Temperature'],
            name='Ambient Temperature',
            line=dict(width=3)
        ))
        fig.add_trace(go.Scatter(
            x=train_data.index,
            y=train_data[tag_value],
            name=tag_value,
            marker=dict(color='#32CD32'),
            line=dict(width=3)
        ))

        fig.update_layout(
            plot_bgcolor='white',
            paper_bgcolor='white',
            xaxis=dict(
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True,
                ticks='outside',
                tickfont=dict(
                    family='Arial',
                    size=20,
                    color='black'
                ),
                showgrid=True,
                gridcolor='lightgray',
                gridwidth=1
            ),
            yaxis=dict(
                showline=True,
                linecolor='black',
                linewidth=1,
                mirror=True,
                ticks='outside',
                tickfont=dict(
                    family='Arial',
                    size=20,
                    color='black'
                ),
                showgrid=True,
                gridcolor='lightgray',
                gridwidth=1
            ),
            font=dict(size=22),
            #height=620,
            #'height': 'auto',
             # 'box-sizing': 'border-box'
        )
        return fig


####### Initialization #################################
if __name__ == '__main__':
    app.run_server(port=8056)
    